In [11]:
# Example response
prompt = """
<s>[INST] 

Role:
- You are an assessor of the Cambridge B2 First English Assessment. You are an expert in this with several years of experience.
- You will be given a conversation between an Examiner and a Candidate and your task is to give scores for two metrics for the responses given by the "Candiate" in the conversation.

Evaluation Steps:

- Read the conversation between the Examiner and the Candidate carefully.
- Assign a score for GRAMMAR_AND_VOCABULARY and  DISCOURSE_MANAGEMENT on a scale of 1 to 5, where 1 is the lowest and 5 is the highest.
- Please disregard the response provided by "Examiner" in your evaluation.
- Present the evaluation categories and scores in JSON format and name it OUTPUT and the OUTPUT will have two key value pairs. GRAMMAR_AND_VOCABULARY and DISCOURSE_MANAGEMENT.
____________

Conversation:[{"Examiner": "What do you usually do during Pongal?", "Candidate": "Eat sweets. See family."}, {"Examiner": "Tell me about being a plumber.", "Candidate": "Fix pipes. Use tools."}, {"Examiner": "Do you like your area?", "Candidate": "Yes, good shops."}, {"Examiner": "What's your hobby?", "Candidate": "I read books."}]

 [/INST]'GRAMMAR_AND_VOCABULARY': 1, 'DISCOURSE_MANAGEMENT': 1767

____________

Conversation:[{"Examiner": "How do you celebrate Makar Sankranti?", "Candidate": "Fly kites. Eat sweets."}, {"Examiner": "Describe being a teacher.", "Candidate": "Teach kids. Grade papers."}, {"Examiner": "Do you like your neighbourhood?", "Candidate": "Yes, has good schools."}, {"Examiner": "What's your favorite pastime?", "Candidate": "I enjoy reading."}]
"""

In [12]:
import re

def extract_values(prompt):
    # Define the pattern to find the values following [/INST]
    pattern = r"\[/INST\](.*?)\n"
    
    # Find all matches of the pattern in the response
    matches = re.findall(pattern, prompt, re.DOTALL)
    
    # Extract and clean the desired values
    extracted_values = []
    for match in matches:
        # Split the string by comma and strip whitespace
        values = [value.strip() for value in match.split(',')]
        # Filter out empty strings and unwanted data
        values = [value for value in values if value.startswith(("'GRAMMAR_AND_VOCABULARY'", "'DISCOURSE_MANAGEMENT'"))]
        # Convert the list of strings to a dictionary
        values_dict = {item.split(":")[0].strip("'"): int(item.split(":")[1].strip()) for item in values}
        extracted_values.append(values_dict)

    return extracted_values

# Extract the values
extracted_values = extract_values(prompt)

# Print the extracted values
for values in extracted_values:
    print(values)

{'GRAMMAR_AND_VOCABULARY': 1, 'DISCOURSE_MANAGEMENT': 1767}


In [17]:
ls

CEFR_Paper1/       i-part2-train.csv  test.ipynb


In [20]:
import pandas as pd
import json

# Load the CSV file
df = pd.read_csv('i-part2-test.csv')

# Function to remove the first four key-value pairs
def remove_first_four(json_string):
    try:
        # Parse the JSON string
        data = json.loads(json_string)
        
        # Check if it's a list and has at least one item
        if isinstance(data, list) and len(data) > 0:
            # Remove the first four key-value pairs
            modified_data = [{k: v for k, v in list(d.items())[4:]} for d in data]
            # Convert back to JSON string
            return json.dumps(modified_data)
    except json.JSONDecodeError:
        # Return the original string if it's not a valid JSON
        return json_string

# Apply the function to the 'text' column
df['PROMPT'] = df['PROMPT'].apply(remove_first_four)

# Save the modified DataFrame to a new CSV file
df.to_csv('i-part2-test-modified.csv', index=False)